PART 1

In [18]:
import csv
import math

# Load Data using built-in csv module
def load_data(filename):
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        data = list(reader)
        # Get the column order from the CSV header
        fieldnames = reader.fieldnames
    return data, fieldnames

# Entropy Calculation without NumPy
def entropy(y):
    # Count frequency of each unique value in y
    freq = {}
    for value in y:
        freq[value] = freq.get(value, 0) + 1
    total = len(y)
    ent = 0.0
    for count in freq.values():
        p = count / total
        ent -= p * math.log2(p)
    return ent

# Information Gain without pandas or NumPy
def information_gain(data, feature, target):
    # Compute the entropy for the target column
    target_values = [row[target] for row in data]
    total_entropy = entropy(target_values)
    
    # Calculate the weighted entropy for each unique value in the feature
    total_count = len(data)
    weighted_entropy = 0.0
    unique_values = set(row[feature] for row in data)
    for value in unique_values:
        subset = [row for row in data if row[feature] == value]
        prob = len(subset) / total_count
        subset_target = [row[target] for row in subset]
        weighted_entropy += prob * entropy(subset_target)
        
    return total_entropy - weighted_entropy

# Helper function to return the majority class for a dataset
def majority_class(data, target):
    freq = {}
    for row in data:
        value = row[target]
        freq[value] = freq.get(value, 0) + 1
    # Return the target value with the highest frequency
    return max(freq, key=freq.get)

# Decision Tree class without using third-party libraries
class DecisionTree:
    def __init__(self, threshold=0.00001):
        self.tree = None
        self.threshold = threshold

    def build_tree(self, data, features, target):
        # Base condition: if all target values are the same, return that value
        target_values = [row[target] for row in data]
        if len(set(target_values)) == 1:
            return target_values[0]
        
        # If no features remain, return the majority class
        if not features:
            return majority_class(data, target)
        
        # Select the best feature based on information gain
        best_feature = max(features, key=lambda f: information_gain(data, f, target))
        ig = information_gain(data, best_feature, target)
        
        # If the information gain is too low, return the majority class
        if ig < self.threshold:
            return majority_class(data, target)
        
        # Create a subtree for each unique value of the best feature
        tree = {best_feature: {}}
        unique_values = set(row[best_feature] for row in data)
        for value in unique_values:
            subset = [row for row in data if row[best_feature] == value]
            remaining_features = [f for f in features if f != best_feature]
            subtree = self.build_tree(subset, remaining_features, target)
            tree[best_feature][value] = subtree
            
        return tree

    def fit(self, data, target, features):
        self.tree = self.build_tree(data, features, target)

    def print_tree(self, tree=None, indent=""):
        if tree is None:
            tree = self.tree
        if isinstance(tree, dict):
            for key, subtree in tree.items():
                print(indent + str(key))
                self.print_tree(subtree, indent + "  ")
        else:
            print(indent + "-> " + str(tree))

# Train and print decision tree for each dataset
datasets = ["rtg_A.csv", "rtg_B.csv", "rtg_C.csv"]

for filename in datasets:
    data, fieldnames = load_data(filename)
    # Assume the target is the last column in the CSV
    target = fieldnames[-1]
    features = fieldnames[:-1]
    print("\nDecision Tree for {}:".format(filename))
    dt = DecisionTree()
    dt.fit(data, target, features)
    dt.print_tree()



Decision Tree for rtg_A.csv:
att0
  0
    att1
      0
        -> 1
      1
        att2
          0
            -> 1
          1
            -> 0
  1
    -> 1

Decision Tree for rtg_B.csv:
att3
  0
    att0
      0
        att2
          0
            -> 1
          1
            att1
              0
                att4
                  0
                    -> 0
                  1
                    -> 1
              1
                -> 1
      1
        att2
          0
            att1
              0
                -> 1
              1
                -> 0
          1
            att1
              0
                att4
                  0
                    -> 0
                  1
                    -> 1
              1
                -> 1
  1
    att4
      0
        att2
          0
            att1
              0
                att0
                  0
                    -> 0
                  1
                    -> 1
              1
                -> 1
    